In [1]:
import numpy as np
import pandas as pd
import pybedtools

import scanpy as sc
import scvelo as scv
scv.settings.verbosity = 0


import warnings
warnings.filterwarnings("ignore")

In [2]:
path = '/data/nelkazwi/code/UniTVelo-ATAC/notebooks/data/HSPC_dataset/'
adata_rna = sc.read(path+"adata_rna_prepro.h5ad")
adata_atac = sc.read(path+"adata_atac_prepro.h5ad")

In [3]:
min_regions = 800
sc.pp.filter_genes(adata_atac, min_counts=min_regions)
sc.pp.filter_cells(adata_atac, min_counts=1)
adata_atac

AnnData object with n_obs × n_vars = 11605 × 93383
    obs: 'celltype', 'n_counts'
    var: 'chrom', 'chromStart', 'chromEnd', 'n_counts'

In [4]:
adata_atac.var

,chrom,chromStart,chromEnd,n_counts
chr1:181070-181803,chr1,181070,181803,1721.0
chr1:191121-192037,chr1,191121,192037,813.0
chr1:267568-268459,chr1,267568,268459,867.0
chr1:629500-630394,chr1,629500,630394,2781.0
chr1:633580-634477,chr1,633580,634477,14154.0
...,...,...,...,...
KI270721.1:2095-2980,KI270721.1,2095,2980,833.0
KI270726.1:41475-42286,KI270726.1,41475,42286,1095.0
KI270713.1:21418-22293,KI270713.1,21418,22293,13315.0
KI270713.1:29583-30478,KI270713.1,29583,30478,3828.0


In [5]:
adata_rna.var

,Accession,chrom,chromEnd,chromStart,Strand
Gene,,,,,
BX004987.1,ENSG00000278704,chrGL000009.2,58376,56140,-
AC145212.1,ENSG00000277400,chrGL000194.1,115018,53590,-
MAFIP,ENSG00000274847,chrGL000194.1,115055,53594,-
AC011043.1,ENSG00000276256,chrGL000195.1,49164,42939,-
AC011043.2,ENSG00000278198,chrGL000195.1,179372,173872,+
...,...,...,...,...,...
TTTY4B,ENSG00000235412,chrY,24607025,24570202,+
BPY2B,ENSG00000183795,chrY,24639207,24607560,+
DAZ4,ENSG00000205916,chrY,24907040,24833843,+


In [15]:
w = 10000
col=['chrom', 'chromStart', 'chromEnd']

ngenes = adata_rna.X.shape[1]
nregions = adata_atac.X.shape[1]
    
gene_coor = adata_rna.var[[col[0], col[1], col[2]]]
gene_coor["gene_name"] = gene_coor.index
gene_coor["gene_number"] = np.arange(0, ngenes)


region_coor = adata_atac.var[[col[0], col[1], col[2]]]
region_coor["region_number"] = np.arange(0, nregions)
adata_atac.var['region_number'] = np.arange(0, nregions) 

a = pybedtools.BedTool.from_dataframe(region_coor)
b = pybedtools.BedTool.from_dataframe(gene_coor)
df_rg_intersection = a.window(b, w=w).overlap(cols=[2, 3, 6, 7])

df_rg_intersection = pd.DataFrame([feature.fields for feature in df_rg_intersection])

df_rg_intersection.to_csv(path +'df_rg_intersections.csv',sep='\t')

In [16]:
df_rg_intersection

,0,1,2,3,4,5,6,7,8,9
0,chr1,181070,181803,0,chr1,141474,173862,AL627309.5,33,-7208
1,chr1,267568,268459,2,chr1,266855,268655,AP006222.2,1714,891
2,chr1,778284,779194,5,chr1,778747,810065,LINC01409,1718,447
3,chr1,827058,827921,6,chr1,826206,827522,LINC00115,36,464
4,chr1,827058,827921,6,chr1,817371,819837,FAM87B,1719,-7221
...,...,...,...,...,...,...,...,...,...,...
128710,chrY,19744370,19745308,93348,chrY,19744756,19759978,AC010889.2,36581,552
128711,chrY,20575239,20576156,93350,chrY,20464916,20575519,TTTY10,36521,280
128712,chrY,20575239,20576156,93350,chrY,20575776,20593154,EIF1AY,36582,380
128713,chrY,20740375,20741262,93351,chrY,20725331,20734526,AC009494.2,36522,-5849
